1. 환경설정 및 라이브러리 import

In [1]:
# 1. 환경 설정 및 필수 라이브러리 임포트
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# API 키 로드 (.env 파일에 설정되어 있어야 합니다)
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. 다중 PDF 문서 로드

In [2]:
file_paths = [
    "C:/Users/user/Downloads/삼성_2025Q4_conference_eng_presentation.pdf",
    "C:/Users/user/Downloads/삼성_2025Q4_script_eng_AudioScript.pdf"
]

docs = []

for path in file_paths:
    # 각 파일을 로드
    loader = PyPDFLoader(path)
    
    # load를 통해 문서의 각 페이지를 Document 객체로 변환
    docs.extend(loader.load())

print(f"총 로드된 페이지 수: {len(docs)}")
# AudioSciprt의 페이지 수: 34
# presentation의 페이지 수: 15
# 총합 로드 페이지 수: 49

총 로드된 페이지 수: 49


3. Text splitting

In [3]:
# 텍스트 1000자 단위로 자르고, 문맥 유지를 위해 50자씩 곂치도록 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

# 로드된 docs (presentation + audio transcript) 분할
splits = text_splitter.split_documents(docs)

print(f"분할된 청크 수: {len(splits)}\n")

# 삼성 presentation의 첫번째 슬라이드 text
print(f"첫번째 청크 예시:\n{splits[0].page_content}\n")

# 삼성 presentation의 두번째 슬라이드 text
print(f"두번쨰 청크 예시:\n{splits[1].page_content}")



분할된 청크 수: 92

첫번째 청크 예시:
SAMSUNG 
ELECTRONICS
Earnings Presentation: 
4Q 2025 Financial Results

두번쨰 청크 예시:
The financial information in this document are consolidated earnings results based on K-IFRS.
This document is provided for the convenience of investors only before the external audit on our 4Q 2025 financial results iscompleted. 
The Audit outcomes may cause some parts of this document to change. 
This document contains "forward-looking statements" - that is statements related to future not past events. 
In this context "forward-looking statements" often address our expected future business and financial performance 
and often contain words such as "expects" "anticipates" "intends" "plans" "believes" "seeks" or "will". 
"Forward-looking statements" by their nature address matters that are to different degrees uncertain. 
For us particular uncertainties which could adversely or positively affect our future results include:
·  The behavior of financial markets including fluctuatio

4. Vector 

허깅페이스 모델중 무료 임베딩 모델:
- "all-MiniLM-L6-v2"
- "BAAI/bge-m3"
- "intfloat/e5-large-v2"
- "intfloat/e5-base-v2"


In [4]:
# gpu 사용가능여부 확인
# cpu 사용시 large csv 파일을 읽을때 오래 걸림

'''
False일시:
1. 기존 버전 삭제:
pip uninstall torch torchvision torchaudio

2. CUDA 지원 버전 설치 (12.1 버전 기준):
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
'''

import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

2.12.0.dev20260218+cu128
True
NVIDIA GeForce RTX 5060 Ti
12.8


In [5]:
# 허깅페이스 오픈소스 무료 임베딩 모델 활용 + GPU 활용
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cuda'}
)

# 분할된 텍스트를 벡터 저장소에 인덱싱하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# Vectorstore에 있는 저장된 벡터를 검색기로 변환하여 출력준비
retriever = vectorstore.as_retriever()

5. LLM 연결

허깅페이스 모델중 무료 LLM 모델:
1. meta-llama/Meta-Llama-3.1-8B-Instruct
2. Qwen/Qwen3-8B
3. mistralai/Mistral-7B-Instruct-v0.3
4. microsoft/Phi-4-mini-instruct
5. google/gemma-3-4b-it
6. google/gemma-2-9b-it

In [17]:
repo_id = "google/gemma-2-9b-it"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=hf_token,
)

chat_llm = ChatHuggingFace(llm=llm_endpoint)

# 프롬프트 설정
template = """
당신은 삼성전자 실적발표 전문 AI 어시스턴트입니다.
제공된 프레젠테이션 자료와 오디오 트랜스크립트 내용을 바탕으로 질문에 대해 상세히 답변하세요.
내용을 요약할 때는 수치(숫자)를 정확히 포함하여 디테일하게 설명하세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

6. RAG 체인 구성

In [18]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_llm
    | StrOutputParser()
)

7. testing

In [20]:
question = "2025 4Q highlights에 대해 최대한 상세하게 알려줘"
response = rag_chain.invoke(question)

print(f"질문: {question}")
print("-" * 30)
print(f"답변: {response}")

질문: 2025 4Q highlights에 대해 최대한 상세하게 알려줘
------------------------------
답변: 2025년 4분기 (4Q25) 주요 성과는 다음과 같습니다.

* **매출:** 93.8조 KRW
* **영업이익:** 20.1조 KRW

**연간 성과:**

* **매출:** 333.6조 KRW
* **영업이익:** 43.6조 KRW

**기타 주요 정보:**

* 2024년 3분기 대비 4분기 매출은 12.2% 증가했습니다.
* 2024년 4분기 대비 4분기 영업이익은 20.1% 증가했습니다.





8. langsmith를 통해 document tracking 및 테스팅

In [21]:
from langchain_teddynote.messages import stream_response

In [ ]:
question = "2025 4Q highlights에 대해 최대한 상세하게 알려줘"
answer = rag_chain.invoke(question)

stream_response(answer)
# langsmith trace: https://smith.langchain.com/public/7bd886a6-f6f1-446d-862e-34db3a014b85/r

2025년 4분기 (4Q25) 주요 성과는 다음과 같습니다.

* **매출:** 93.8조원
* **영업이익:** 20.1조원

**연간 성과:**

* **매출:** 333.6조원
* **영업이익:** 43.6조원

**분기별 성과:**

* 2024년 4분기: 매출 333.6조원, 영업이익 6.6조원
* 2025년 3분기: 매출 258.9조원, 영업이익 12.2조원
* 2025년 4분기: 매출 93.8조원, 영업이익 20.1조원




In [ ]:
question = "HBM4 개발 현황이랑 2026년 HBM 매출 전망 알려줘"
answer = rag_chain.invoke(question)

stream_response(answer)
# langsmith trace: https://smith.langchain.com/public/51990f59-544d-4a05-b580-b037200f31e6/r

## HBM4 개발 현황 및 2026년 HBM 매출 전망

**HBM4 개발 현황:**

* 삼성은 HBM4 개발 초기부터 JEDEC 표준을 뛰어넘는 고성능 목표를 설정했습니다.
* 주요 고객의 성능 요구 사항이 높아지더라도, 재설계 없이 샘플을 공급하고 최종 자격 검증 단계에 진입했습니다.
* 고객으로부터 HBM4의 경쟁력 있는 성능에 대한 긍정적인 피드백을 받고 있습니다.
* 현재 HBM4는 11.7 Gbps pin-speed를 가진 최고 성능 Bin을 포함하여 안정적으로 대량 생산되고 있으며, 2월부터 출하가 시작됩니다.

**2026년 HBM 매출 전망:**

* 2026년 HBM 생산 용량은 현재 고객 주문으로 완전히 확보되어 있습니다.
* 2026년 HBM 매출은 전년 대비 **3배 이상 증가**할 것으로 예상됩니다.
* 2026년에도 고객의 HBM 수요가 삼성의 공급량을 넘어서는 것으로 예상됩니다.




In [ ]:
question = "2025년 4분기에 DS사업부는 좋아졌는데 DX사업부는 왜 나빠졌어?"
answer = rag_chain.invoke(question)

stream_response(answer)
# langsmith trace: https://smith.langchain.com/public/e5f89ff3-f51e-44aa-bf9e-c2cdf73f3eee/r

2025년 4분기에 DS 사업부는 좋아졌지만, DX 사업부는 새로운 스마트폰 출시 효과가 사라지고 미국 관세로 인해 가전제품 판매가 부진하여 8% sequentially 감소했습니다. 


